In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [3]:
import os
import PIL
import keras
import tensorflow as tf

In [4]:
os.getcwd()

In [5]:
path = '../input/siim-isic-melanoma-classification/jpeg/train'

In [6]:
len(os.listdir(path))

In [8]:
df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

In [13]:
df.head(10)

In [10]:
df.shape

In [29]:
pd.value_counts(df['target'])

In [11]:
df['benign_malignant'].unique()

In [12]:
pd.value_counts(df['benign_malignant'])

In [14]:
from PIL import Image

In [16]:
img = Image.open('../input/siim-isic-melanoma-classification/jpeg/train/ISIC_0015719.jpg')

In [18]:
img = np.asarray(img)

In [20]:
plt.imshow(img)

In [26]:
images = os.listdir(path)
plt.figure(figsize=(10,6))
for i in range(50):
    img = Image.open(path + '/' + images[i])
    img = np.asarray(img)
    plt.subplot(10,5,i+1)
    plt.title(df['benign_malignant'][i])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
plt.show()

In [27]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [28]:
# 3 ways to pass data in data generator
# - flow
# - flow from directory
# - flow from data frame

In [34]:
images[0]

In [55]:
train_path = '../input/siim-isic-melanoma-classification/jpeg/train'
train_images = os.listdir(train_path)

In [56]:
train_images[:5]

In [57]:
df['image_name'][:5]

In [51]:
# data_dictionary = {'img_path' : [], 'target' : []}
# for i in range(len(images)):
#     for j in range(len(df['image_name'])):
#         if images[i].startswith(df['image_name'][j]):
#             #print(images[i],"::",df['image_name'][j])
#             data_dictionary['img_path'].append(path + "/" + images[i])
#             data_dictionary['target'].append(df['target'][i])
#             break

In [58]:
train_images.sort()

In [53]:
df.sort_values('image_name', inplace=True)

In [59]:
train_images[:5]

In [60]:
df['image_name'][:5]

In [80]:
data_dictionary = {'img_path' : [], 'target' : []}
for i in range(len(images)):
    data_dictionary['img_path'].append(path + "/" + images[i])
    data_dictionary['target'].append(df['benign_malignant'][i])

In [81]:
final_df = pd.DataFrame(data_dictionary)

In [82]:
final_df.head()

In [83]:
from sklearn.model_selection import train_test_split

In [84]:
x_train, x_test, y_train, y_test = train_test_split(final_df['img_path'], final_df['target'], test_size=0.25)

In [85]:
train_df = pd.DataFrame(x_train)

In [86]:
train_df['target'] = y_train

In [87]:
train_df.head()

In [88]:
test_df = pd.DataFrame(x_test)
test_df['target'] = y_test

In [89]:
train_gen = train_datagen.flow_from_dataframe(train_df, 
                                              x_col='img_path',
                                              y_col='target',
                                              target_size=(299,299),
                                              batch_size=256,
                                              class_mode='binary',
                                             )

test_gen = test_datagen.flow_from_dataframe(test_df,
                                            x_col='img_path',
                                            y_col='target',
                                            target_size=(299,299),
                                            shuffle=False,
                                            batch_size=256,
                                            class_mode='binary',
                                           )

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [91]:
model = Sequential([
    Conv2D(32, 3, input_shape=(299,299,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [92]:
model.summary()

In [93]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [94]:
model_fit = model.fit(train_gen, epochs=10, validation_data=test_gen)